In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/distance/Distance_Matrix.csv
/kaggle/input/biomass-data/biomass.csv
/kaggle/input/biomass/Biomass_History.csv


In [2]:
!pip install keras-rl2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 1.8 MB/s eta 0:00:00


In [5]:
import plotly.express as px
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredLogarithmicError,MeanSquaredError,MeanAbsoluteError
from matplotlib import pyplot as plt
from shapely.geometry import Point, MultiPoint
import rtree
from sklearn.neighbors import NearestNeighbors
from gym import Env
from gym.spaces import Box, MultiDiscrete, Dict,Discrete
from shapely.ops import nearest_points
import geopandas as gpd
import seaborn as sns
import random
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
import bisect
%matplotlib inline

In [7]:
biomass = pd.read_csv("/kaggle/input/biomass/Biomass_History.csv")
distance = pd.read_csv("/kaggle/input/distance/Distance_Matrix.csv")
# creating point objects, will be useful later on 
points = []
for i, row in biomass.iterrows():
    points.append((float(row['Longitude']),float(row["Latitude"])) )

biomass["Geometries"] = points
biomass["Type"] = "Biomass prod.plants"
print(biomass.head())


   Index  Latitude  Longitude       2010       2011       2012       2013  \
0      0  24.66818   71.33144   8.475744   8.868568   9.202181   6.023070   
1      1  24.66818   71.41106  24.029778  28.551348  25.866415  21.634459   
2      2  24.66818   71.49069  44.831635  66.111168  56.982258  53.003735   
3      3  24.66818   71.57031  59.974419  80.821304  78.956543  63.160561   
4      4  24.66818   71.64994  14.653370  19.327524  21.928144  17.899586   

        2014       2015       2016        2017            Geometries  \
0  10.788374   6.647325   7.387925    5.180296  (71.33144, 24.66818)   
1  34.419411  27.361908  40.431847   42.126945  (71.41106, 24.66818)   
2  70.917908  42.517117  59.181629   73.203232  (71.49069, 24.66818)   
3  93.513924  70.203171  74.536720  101.067352  (71.57031, 24.66818)   
4  19.534035  19.165791  16.531315   26.086885  (71.64994, 24.66818)   

                  Type  
0  Biomass prod.plants  
1  Biomass prod.plants  
2  Biomass prod.plants  
3  B

In [ ]:
sns.heatmap(biomass.corr(), annot=True)
plt.show()

In [ ]:
fig = px.scatter_mapbox(biomass, lat="Latitude", lon="Longitude",hover_data=["Index"],
                        color="2014", zoom=3, height=400)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show(renderer='iframe')

In [ ]:
#predictions using neural networks

""" First a little data prep. We need to batch the results from 2010 till 2016, the 2017 
results will serve as the target values for the neural network"""

x = biomass[[str(i+2000) for i in range(10,17)]].values
print(x.shape)
y = biomass["2017"].values
print(y.shape)
scaler = StandardScaler()
x = scaler.fit_transform(x)
X_train,X_test,Y_train,Y_test = train_test_split(x,y, test_size = 0.25, random_state = 32)
print (X_train.shape, X_test.shape)


def build_net_no_dropout(input_size):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(input_size,activation = 'relu'))
    model.add(tf.keras.layers.Dense(128, activation = "relu"))
    model.add(tf.keras.layers.Dense(32, activation = "relu"))
    model.add(tf.keras.layers.Dense(1, activation = 'relu'))
    return model


def build_net_with_dropout(input_size):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(input_size,activation = 'relu',kernel_regularizer = tf.keras.regularizers.L1(0.01)))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(128, activation = "relu",kernel_regularizer = tf.keras.regularizers.L1(0.01)))
    model.add(tf.keras.layers.Dropout(0.7))
    model.add(tf.keras.layers.Dense(32, activation = "relu",kernel_regularizer = tf.keras.regularizers.L1(0.01)))
    model.add(tf.keras.layers.Dense(1, activation = 'relu'))
    return model

def model_run(model,train_x,val_y, train_y, val_x, num_epochs):
    loss = MeanSquaredLogarithmicError()
    optimizer = "adam"
    model.compile( loss= loss, optimizer = optimizer, metrics = ["msle"])
    #training
    history = model.fit(train_x,train_y, epochs = num_epochs,batch_size= 70, verbose = 1,validation_data = (val_x,val_y))
    return history

def plotter(history,metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric])
    plt.xlabel('Epochs')
    plt.ylabel(metric)
    plt.legend([metric,'val_'+metric])
    plt.show()
    



In [ ]:
#putting it all together

ann_without = build_net_no_dropout(X_train.shape[1])
hist_ann_without = model_run(ann_without,X_train,Y_test,Y_train,X_test,30)
plotter(hist_ann_without, "msle")
plotter(hist_ann_without, "loss")


In [ ]:
ann_with = build_net_with_dropout(X_train.shape[1])
hist_ann_with = model_run(ann_with,X_train,Y_test,Y_train,X_test,40)
plotter(hist_ann_with, "msle")
plotter(hist_ann_with, "loss")

In [ ]:

Predictions = ann_without.predict(X_test)
Real = Y_test
dft = pd.DataFrame({"real_2017":list(Y_test),"Prediction_2017":list(Predictions)})
plt.figure(figsize =(20,4))
plt.plot(dft["real_2017"])
plt.plot(dft["Prediction_2017"])
plt.legend(["Real", "Predictions"])
plt.show()

In [66]:

#(number,name, k, cap,pointlist,depots,pointx=[],index=-1,refinery=False, allx=points.copy()

depots,pointx = make_components(25,"depot_location",80, 20000,pointx =points.copy())
dep_points = [q.point for q in depots]
dep_index =[points.index(q)for q in dep_points]
refineries,unused_depots,refpointx = make_refinery(5,"refinery_location",5, 100000,dep_index.copy(),pointx)
usage(points,depots,refineries)    

2417
2337
2336
2256
2255
2222
2221
2141
2140
2060
2059
1979
1978
1898
1897
1817
1816
1736
1735
1655
1654
1574
1573
1493
1492
1421
1420
1340
1339
1259
1258
1178
1177
1097
1096
1016
1015
935
934
854
853
807
806
726
725
663
662
612
611
531
done
530
529
528
527
526
done


NameError: name 'usage' is not defined

In [61]:
len(pointx)

2291

In [67]:
# each additional component in the waste to energy supply chain will be represented 

class Component():
    def __init__(self, index, name, num, cap,k,pointx,refinery, data=biomass.copy(),distance = distance.copy(),spare =points.copy(),sources=None):
        self.name = name
        self.index = index
        self.num = num       
        self.capacity = cap
        self.is_ref = refinery
        self.tree = rtree.index.Rtree()
        self.point = spare[index]
        
        self.pointx = pointx.copy()
            
    
        if refinery==True and sources!=  None:
            self.depot_left = sources.copy()
            for i,a in enumerate(sources):# sources should be index numbers
                p=spare[a]
                self.tree.insert(i,p+p,p)
                
        else:
            for i,p in enumerate(self.pointx):
                self.tree.insert(i,p+p,p)
            
       
        #print(len(self.pointx)) 
        
        # k nearest point
        self.sources= list(self.tree.nearest(self.point, k+1,objects='raw' ))
        if len(self.sources)>k+1:
            self.sources = self.sources[:k+1]
        
        self.sources = self.sources.copy()[1:]      
        #print(len(self.sources))
        self.sources_index = []
        if self.is_ref ==False:
            for i in self.sources:
                self.sources_index.append(spare.index(i))
            
        else: 
            for i in self.sources:
                self.sources_index.append(spare.index(i))
               
            
        #print("Original number of sources {}".format(len(self.sources)))
        
        #this part calculates the total output of the sources
        if self.is_ref==False:
            sources_output = data["2017"][self.sources_index].sum()
        
            self.under_util = 0
            if sources_output>self.capacity and self.is_ref == False:
                while True:
                    lee = data["2017"][self.sources_index].values
                    minx = np.argmin(lee, axis = 0)
                    self.sources.pop(minx)
                    self.sources_index.pop(minx)
                    sources_output = data["2017"][self.sources_index].sum()
                
                    if sources_output<=self.capacity:
                        self.under_util = self.capacity-sources_output
                        break
                    else:
                        continue
                
            if self.is_ref==False:    
                for i in self.sources:
                    self.pointx.remove(i)
                
            
                
            #print(len(self.pointx))        
            self.sources_output = sources_output 
        
         
    
        
        self.sources_index = [spare.index(i) for i in self.sources.copy()]
        self.total_dist = 0
        for i in self.sources_index:
            self.total_dist+=distance.iloc[i,self.index]
        
        #print("final number of sources {}".format(len(self.sources)))
        
    def prevent_duplicates(self):
        if self.is_ref ==True:
            depot_left = self.depot_left
            for iw in self.sources_index:
                depot_left.remove(iw)
            del(self.depot_left)
            return self.pointx,depot_left.copy()
        else:
            return self.pointx
            
    def add(self,df):
        q= int(self.index)
        df.Type[q] = str(self.name)
        return df
    def output(self):
        return self.sources_output
    def cost(self):
        if self.is_ref ==True:
            return self.total_dist
        else:
            return self.total_dist,self.under_util
    
            
        
        

def make_refinery(number,name, k, cap,depots,pointx,index=-1,refinery=True, allx=points.copy()):
    
    refineries = [] 
    if index==-1:        
        listr = [] 
        
        for i in range(number):
            index = allx.index(random.choice(pointx))
            
            if index not in listr :
                pointx.remove(allx[index])
                
                #print("current avail dept: {}".format(depots))
                comp = Component(index,name,i,cap,k,pointx,refinery = refinery,sources=depots.copy())
                pointx,depots= comp.prevent_duplicates()
                refineries.append(comp)
                listr.append(index)
                
            else:
                i = i-1
                continue
    print("done")
    return refineries.copy(),depots.copy(),pointx.copy()   

def make_components(number,name, k, cap,pointx,index=-1,refinery=False, allx = points.copy()):
    components = [] 
    if index==-1:        
        listr = [] 
        for i in range(number):
            index = allx.index(random.choice(pointx))
            if index not in listr:
                #print (pointx)
                pointx.remove(allx[index])
                comp = Component(index,name,i,cap,k,pointx,refinery = refinery)
                pointx= comp.prevent_duplicates()
                components.append(comp)
                listr.append(index)
                
            else:
                i = i-1
                continue
    print("done")
    return components.copy(),pointx.copy()
        
    
    
     
    
    


In [ ]:
q= np.array([1,4])
q.shape


In [87]:
#creating a custom environment with gym

class Environment(Env):
    def __init__(self,num_dep,num_refinery, points = points):
        a = 0.001
        b = 1
        c = 1
        self.k = 80
        self.points = points.copy() 
        self.num_dep = num_dep
        self.refinery_cap = 100000
        self.num_refinery = num_refinery
        self.dep_cap = 20000
        #new
        
        self.depots,self.pointx = make_components(self.num_dep,"depot_location",self.k, self.dep_cap,self.points.copy())
        print(len(self.pointx))
        self.dep_points = [q.point for q in self.depots]
        self.dep_index =[points.index(q)for q in self.dep_points]
        #(number,name, k, cap,depots,pointx,index=-1,refinery=True, allx=points.copy())
        self.refineries,self.unused_depots,self.pointx = make_refinery(self.num_refinery,"refinery_location",5, 100000,self.dep_index.copy(),self.pointx.copy())
        print(len(self.pointx)) 
        #new
        self.ref_points = [q.point for q in self.refineries]
        self.transport = 0
        self.underutil = 0
        self.ndim=(self.num_dep+self.num_refinery-1,2)
        self.observation_space = Box(high= np.inf,low = -np.inf,shape = (2,))
        self.action_space =Discrete(np.prod(self.ndim))
        all_dep_output = [q.sources_output for q in self.depots.copy()]
        all_dep_indexes = [q.index for q in self.depots.copy()]
        for i in self.depots:
            tr,un = i.cost()
            self.transport+=tr
            self.underutil+=un
            
        for i in self.refineries:
            use = 0
            for b in i.sources_index:
                use+= all_dep_output[all_dep_indexes.index(b)]
            un = 100000-use
            tr = i.cost()
            self.transport+=tr
            self.underutil+=un
        self.state = (a*self.transport)+(c*self.underutil)
        self.episode_len = 100
        self.use = self.usage()
        self.previous_score = []
        self.reinitialize= {"depots":self.depots.copy(),
                            "refinery":self.refineries.copy(), 
                            "refinery_points":self.ref_points.copy(),
                            "depot_points" :self.dep_points.copy(),
                            "pointx":self.pointx.copy(),
                            'Unused_depots':self.unused_depots,
                            "usage" : self.use,
                            "state": self.state
                           }
        print("utilizing {} percent of all available output".format(self.use))
        
        
    def usage(self):
        
        all_dep_output = [q.sources_output for q in self.depots.copy()]
        #print(all_dep_output)
        all_dep_indexes = [q.index for q in self.depots.copy()]
        all_ref_sources = [q.sources_index for q in self.refineries.copy()]
        all_ref_sources_flat=[]
        for e in all_ref_sources:
            for q in e:
                all_ref_sources_flat.append(q)
    
        
        total_usage = 0
        for i in all_ref_sources_flat:
            if i in all_dep_indexes:
                pla = all_dep_indexes.index(i)
                total_usage= total_usage+all_dep_output[pla]
                #print(total_usage)
        perc = 100*(total_usage/(biomass["2017"].sum()))
        return perc
         

    
            
        
        
            
        
        
    def observe(self):
        return np.array([self.state,self.use])
    
    def reset(self):
        self.reinitialize["depots"] = self.depots
        self.reinitialize["refinery"] = self.refineries 
        self.reinitialize["refinery_points"] =self.ref_points
        self.reinitialize["depot_points"] =self.dep_points
        self.reinitialize["deppointx"] = self.pointx
        self.reinitialize['Unused_depots'] = self.unused_depots
        self.reinitialize["usage"] = self.usage
        self.episode_len = 100
        self.previous_score = []
        self.state = self.recalc()
        obs = self.observe()
        return obs
        
    def recalc (self):
        a = 0.001
        b = 1
        c = 1
        self.transport = 0
        self.underutil = 0
        all_dep_output = [q.sources_output for q in self.depots.copy()]
        all_dep_indexes = [q.index for q in self.depots.copy()]
        for i in self.depots:
            tr,un = i.cost()
            self.transport+=tr
            self.underutil+=un
        for i in self.refineries:
            use = 0
            for b in i.sources_index:
                use+= all_dep_output[all_dep_indexes.index(b)]
            un = 100000-use
            tr = i.cost()

            self.transport+=tr
            self.underutil+=un
        var = (a*self.transport)+(c*self.underutil)
        return var
        
    def close(self):
        pass

    def find_closest(self, target, less=True):
    # Ensure the list is sorted for binary search
        lst = [self.points.index(q) for q in self.pointx.copy()]
        lst.sort()

    # Find the insertion point for the target using bisect
        pos = bisect.bisect_left(lst, target)

    # If target is found in the list, return it
        if pos < len(lst) and lst[pos] == target:
            return lst[pos]

        if less:
        # Find the closest number less than the target
            if pos == 0:
                return None  # No number less than target
            else:
                return lst[pos - 1]  # Return the closest smaller number
        else:
        # Find the closest number greater than the target
            if pos == len(lst):
                return None  # No number greater than target
            else:
                return lst[pos]  # Return the closest larger number



        
    def step(self, action):
        reward = 0
        mapping = tuple(np.ndindex(self.ndim))
        action = mapping[action]
        self.episode_len -= 1 
        if action[0]>=(self.num_dep) and self.episode_len>0:
            # if the agent picks from [0, self.num_dep] that indicates a depot, else its a refinery
            # in this case it is a refinery that has been selected
            selection = action[0]-self.num_dep
            old_ref = self.refineries[selection]
            old_ref_number = old_ref.num
            old_ref_index =old_ref.index
            if action[1] == 0:
                #moves left
                new_ref_index = self.find_closest(old_ref_index-10, less=True)
                if new_ref_index !=None:
                    self.refineries[selection] = None
                    #freeing up the old refinery's attached depots
                    for de in old_ref.sources_index:
                        self.unused_depots.append(de)
                    self.pointx.append(old_ref.point)
                    #Component(index,name,i,cap,k,pointx.copy(),refinery = refinery,sources=depots.copy())
                    new_ref = Component(new_ref_index,"refinery_location",old_ref_number,100000,5,self.pointx.copy(),refinery=True,sources=self.unused_depots.copy())
                    self.pointx.remove(new_ref.point)
                    self.pointx,self.unused_depots = new_ref.prevent_duplicates()
                    self.refineries[selection] =  new_ref
                    self.state= self.recalc()
                else:
                    reward-=4
                
                    
                
            elif action[1] ==1:
                #moves right
                pass

            
        
        
        
        # Calculating the reward
        if len(self.previous_score)>0:
            reward += (20*(self.previous_score[-1]-self.state))/self.previous_score[-1]
            self.previous_score.append(self.state)
 
  
        elif len(self.previous_score)==0:
            self.previous_score.append(self.state) 
            
        # additional rewards
        self.use = self.usage()
        if self.use<=80:
            reward+= 6
        else:
            reward-=2
            
        
        # Checking if episode is done
        if self.episode_len <= 0: 
            done = True
        else:
            done = False
        
        # Setting the placeholder for info
       
        info = {'usage':self.use}
        
        # Returning the step information
        return self.observe(), reward, done,info

In [12]:
a = [2,9,5,6,7]
b =a[2]
a[2]=0
print(b)

5


In [88]:
Enviro = Environment(25,5)
state = Enviro.observation_space.shape
actions = Enviro.action_space.n

done
538
done
533
utilizing 78.82809711111122 percent of all available output


In [70]:
sum([len(i.sources_index)for i in Enviro.depots])

1925

In [ ]:
t = 0
while True:
    
    t += 1
    observation = Enviro.reset()
    print(observation)
    action = Enviro.action_space.sample()
    observation, reward, done, info = Enviro.step(action)
    if done:
        print("Episode finished after {} timesteps".format(t+1))
        break
    Enviro.close()

[1.97284899e+05 7.88280971e+01]
[1.96567715e+05 7.90145049e+01]
[1.96567715e+05 7.90145049e+01]
[1.96567715e+05 7.90145049e+01]
[1.96567715e+05 7.90145049e+01]
[1.96567715e+05 7.90145049e+01]
[1.96567715e+05 7.90145049e+01]
[1.96567715e+05 7.90145049e+01]
[1.96567715e+05 7.90145049e+01]
[1.99017191e+05 7.83781550e+01]
[1.99019211e+05 7.83781550e+01]
[1.99019211e+05 7.83781550e+01]
[1.99019211e+05 7.83781550e+01]
[1.99019211e+05 7.83781550e+01]
[1.99019211e+05 7.83781550e+01]
[1.99019211e+05 7.83781550e+01]
[1.99019211e+05 7.83781550e+01]
[1.99019211e+05 7.83781550e+01]
[1.99019211e+05 7.83781550e+01]
[1.99019211e+05 7.83781550e+01]
[1.99019211e+05 7.83781550e+01]
[1.99019211e+05 7.83781550e+01]
[1.99019211e+05 7.83781550e+01]
[1.99019211e+05 7.83781550e+01]
[1.99019211e+05 7.83781550e+01]
[1.99019211e+05 7.83781550e+01]
[1.99019211e+05 7.83781550e+01]
[1.99019211e+05 7.83781550e+01]
[1.99019211e+05 7.83781550e+01]
[1.99019211e+05 7.83781550e+01]
[1.99019211e+05 7.83781550e+01]
[1.99019

In [23]:
def usage(points,depots,refineries):
         
        all_dep_output = [q.sources_output for q in depots]
        print(all_dep_output)
        all_dep_indexes = [q.index for q in depots]
        all_ref_sources = [q.sources_index for q in refineries]
        all_ref_sources_flat=[]
        for e in all_ref_sources:
            for q in e:
                all_ref_sources_flat.append(q)
       
                                   
        print(all_dep_indexes)
        print("refinery sources \n")
        
        print(all_ref_sources_flat)
    
        
        total_usage = 0
        for i in all_ref_sources_flat:
            if i in all_dep_indexes:
                pla = all_dep_indexes.index(i)
                total_usage= total_usage+all_dep_output[pla]
                print(total_usage)
        perc = 100*(total_usage/(biomass["2017"].sum()))
        return perc

In [22]:
run=89

In [ ]:
[q.index for q in Enviro.refineries]

In [ ]:
 [q.index for q in Enviro.depots]

In [ ]:
 [q.sources_index for q in Enviro.refineries]

In [ ]:
# model
model = tf.keras.models.Sequential()    
model.add(tf.keras.layers.Dense(32, activation='relu', input_shape =(1,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(actions, activation='linear'))


In [ ]:
Enviro.usage

In [ ]:

def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=100, target_model_update=1e-2)
    return dqn

In [ ]:
[points.index(i) for i in Enviro.dep_points]

In [ ]:
benv = Environment(25,5)

In [ ]:
Agent = build_agent(model, actions)
Agent.compile(tf.keras.optimizers.legacy.Adam(learning_rate=1e-2), metrics=['mae'])
Agent.fit(Enviro , nb_steps=1000, visualize=False, verbose=1)